In [1]:
import h5py
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import scatter_matrix
import seaborn as sns

# from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC
# from sklearn.metrics import roc_curve, roc_auc_score, classification_report, confusion_matrix,  ConfusionMatrixDisplay, RocCurveDisplay, PrecisionRecallDisplay
# from sklearn.model_selection import learning_curve
# from sklearn.decomposition import PCA
# from sklearn.tree import plot_tree
# from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_Code15 = pd.read_csv("../data/CODE-15%_exams.csv")
df_Code15 = df_Code15[['exam_id', 'age', 'is_male', 'trace_file']]
df_Code15.insert(4, "mean_ecg", -1)
df_Code15.insert(4, "mode_ecg", -1)
df_Code15.insert(4, "median_ecg", -1)
#df_Code15.insert(4, "folder", )
display(df_Code15)

df_ptbxl = pd.read_csv("../data/ptbxl_database.csv")
df_ptbxl = df_ptbxl[['ecg_id','patient_id', 'age', 'sex','filename_lr', 'filename_hr']]
df_ptbxl.insert(6, "mean_ecg", -1)
df_ptbxl.insert(6, "mode_ecg", -1)
df_ptbxl.insert(6, "median_ecg", -1)
df_ptbxl.insert(6, "folder", "PTB")
display(df_ptbxl)

df_Sami = pd.read_csv("../data/Sami-Trop_exams.csv")
df_Sami = df_Sami[['exam_id', 'age', 'is_male']]
df_Sami.insert(3, "mean_ecg", -1)
df_Sami.insert(3, "mode_ecg", -1)
df_Sami.insert(3, "median_ecg", -1)
df_Sami.insert(3, "folder", "sami")
display(df_Sami)

,exam_id,age,is_male,trace_file,median_ecg,mode_ecg,mean_ecg
0,1169160,38,True,exams_part13.hdf5,-1,-1,-1
1,2873686,73,True,exams_part13.hdf5,-1,-1,-1
2,168405,67,True,exams_part13.hdf5,-1,-1,-1
3,271011,41,True,exams_part13.hdf5,-1,-1,-1
4,384368,73,True,exams_part13.hdf5,-1,-1,-1
...,...,...,...,...,...,...,...
345774,1123951,33,True,exams_part2.hdf5,-1,-1,-1
345775,954704,73,False,exams_part2.hdf5,-1,-1,-1
345776,589697,75,False,exams_part2.hdf5,-1,-1,-1
345777,2780563,44,False,exams_part2.hdf5,-1,-1,-1


,ecg_id,patient_id,age,sex,filename_lr,filename_hr,folder,median_ecg,mode_ecg,mean_ecg
0,1,15709.0,56.0,1,records100/00000/00001_lr,records500/00000/00001_hr,PTB,-1,-1,-1
1,2,13243.0,19.0,0,records100/00000/00002_lr,records500/00000/00002_hr,PTB,-1,-1,-1
2,3,20372.0,37.0,1,records100/00000/00003_lr,records500/00000/00003_hr,PTB,-1,-1,-1
3,4,17014.0,24.0,0,records100/00000/00004_lr,records500/00000/00004_hr,PTB,-1,-1,-1
4,5,17448.0,19.0,1,records100/00000/00005_lr,records500/00000/00005_hr,PTB,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...
21794,21833,17180.0,67.0,1,records100/21000/21833_lr,records500/21000/21833_hr,PTB,-1,-1,-1
21795,21834,20703.0,300.0,0,records100/21000/21834_lr,records500/21000/21834_hr,PTB,-1,-1,-1
21796,21835,19311.0,59.0,1,records100/21000/21835_lr,records500/21000/21835_hr,PTB,-1,-1,-1
21797,21836,8873.0,64.0,1,records100/21000/21836_lr,records500/21000/21836_hr,PTB,-1,-1,-1


,exam_id,age,is_male,folder,median_ecg,mode_ecg,mean_ecg
0,294669,67,True,sami,-1,-1,-1
1,291318,65,True,sami,-1,-1,-1
2,247007,67,False,sami,-1,-1,-1
3,181629,34,False,sami,-1,-1,-1
4,250434,52,False,sami,-1,-1,-1
...,...,...,...,...,...,...,...
1626,776005,63,False,sami,-1,-1,-1
1627,460507,58,False,sami,-1,-1,-1
1628,395721,80,False,sami,-1,-1,-1
1629,186505,48,False,sami,-1,-1,-1


In [3]:
labels_df = pd.read_csv("../data/Code15/code15_chagas_labels.csv")

records = []

# Define the lead names in the order stored in the HDF5 file.
lead_names = ['DI', 'DII', 'DIII', 'AVR', 'AVL', 'AVF',
              'V1', 'V2', 'V3', 'V4', 'V5', 'V6']

# Open the HDF5 file.
with h5py.File("../data/Code15/exams_part0.hdf5", "r") as f:
    # Load the exam ids (assumed small enough to load into memory)
    exam_ids_hdf5 = f['exam_id'][:]  # shape: (N,)
    # Build a mapping from exam_id (converted to string) to its index.
    exam_id_to_index = {str(eid): idx for idx, eid in enumerate(exam_ids_hdf5)}

    # Access the tracings dataset without converting the whole thing to a NumPy array.
    tracings = f['tracings']  # shape: (N, 4096, 12)

    # Process each exam_id from the CSV.
    for _, row in labels_df.iterrows():
        exam_id = str(row['exam_id'])
        chagas_val = row['chagas']

        # Ensure the exam_id exists in the HDF5 file.
        if exam_id in exam_id_to_index:
            index = exam_id_to_index[exam_id]
            # Read only the required chunk for this exam_id.
            ecg_data = tracings[index, :, :]  # shape: (4096, 12)

            # Create a record for the DataFrame.
            record = {"exam_id": exam_id, "chagas": chagas_val}
            # For each lead, extract the corresponding signal and trim leading/trailing zeros.
            for j, lead in enumerate(lead_names):
                signal = ecg_data[:, j]
                # Find indices of non-zero elements.
                nonzero_indices = np.nonzero(signal)[0]
                if nonzero_indices.size > 0:
                    start = nonzero_indices[0]
                    end = nonzero_indices[-1] + 1  # +1 to include the last non-zero element.
                    trimmed_signal = signal[start:end]
                else:
                    # If the signal is all zeros, leave it unchanged.
                    trimmed_signal = signal
                record[lead] = trimmed_signal

            records.append(record)

# Construct a pandas DataFrame from the collected records.
df = pd.DataFrame(records)

# --- Standardize each ECG signal array for each lead ---
# for idx, row in df.iterrows():
#     for lead in lead_names:
#         signal = row[lead]
#         # Compute the mean and standard deviation of the signal.
#         mean_val = np.mean(signal)
#         std_val = np.std(signal)
#         # Avoid division by zero.
#         if std_val != 0:
#             standardized_signal = (signal - mean_val) / std_val
#         else:
#             standardized_signal = signal - mean_val
#         # Update the DataFrame with the standardized signal.
#         df.at[idx, lead] = standardized_signal

# Example: print the first row to see the standardized arrays.
print(df.head())

   exam_id  chagas                                                 DI  \
0  3000862   False  [2.4625041, 2.6512797, 2.2965233, 2.5552232, 2...   
1  2903368   False  [-0.09771842, -0.114296556, -0.05233861, -0.01...   
2  2891944   False  [0.04885921, 0.09299803, 0.048911728, 0.003107...   
3  2872920   False  [0.019543683, 0.004188288, -0.00091002416, -0....   
4  4243597   False  [2.8125315, 2.7956874, 2.7961586, 2.8219056, 2...   

                                                 DII  \
0  [-3.9966831, -4.3118763, -3.7316463, -4.137255...   
1  [-0.068402894, -0.10931933, 0.0883364, 0.28733...   
2  [0.12703393, 0.16206351, -0.07298482, -0.25317...   
3  [0.068402894, 0.088374145, 0.026369374, 0.0638...   
4  [-1.0975733, -1.1380206, -1.174241, -1.1905475...   

                                                DIII  \
0  [-6.459187, -6.9631557, -6.0281696, -6.692478,...   
1  [0.029315524, 0.22461408, 0.29533225, 0.257909...   
2  [0.07817473, -0.15057138, -0.2765538, -0.21380...   


In [4]:
# File paths
hdf5_file_path = "../data/sami/sami_exams.hdf5"
labels_csv_path = "../data/sami/samitrop_chagas_labels.csv"

# Load the labels dataframe and display it first
labels_df = pd.read_csv(labels_csv_path)

# Initialize the records list to store processed data
records = []

# Define lead names in the correct order
lead_names = ['I', 'II', 'III', 'AVR', 'AVL', 'AVF',
              'V1', 'V2', 'V3', 'V4', 'V5', 'V6']

# Open HDF5 file and process data
with h5py.File(hdf5_file_path, 'r') as hdf:
    # Access the tracings dataset
    tracings = hdf['tracings']
    print(f"\nTracings shape: {tracings.shape}")
    
    # Process each row in the labels dataframe
    for idx, row in labels_df.iterrows():
        # Get information from CSV
        exam_id = str(row['exam_id'])
        chagas_val = row['chagas']
        
        # Use the index position to access the corresponding tracing
        ecg_data = tracings[idx, :, :]  # shape: (4096, 12)
        
        # Create a record with exam_id and chagas label
        record = {"exam_id": exam_id, "chagas": chagas_val}
        
        # Process each lead
        for j, lead in enumerate(lead_names):
            signal = ecg_data[:, j]
            
            # Find indices of non-zero elements for trimming
            nonzero_indices = np.nonzero(signal)[0]
            if nonzero_indices.size > 0:
                start = nonzero_indices[0]
                end = nonzero_indices[-1] + 1  # +1 to include the last non-zero element
                trimmed_signal = signal[start:end]
            else:
                # If the signal is all zeros, keep it as is
                trimmed_signal = signal
                
            record[lead] = trimmed_signal
        
        records.append(record)
        
        # # Print progress every 100 records
        # if (idx + 1) % 100 == 0:
        #     print(f"Processed {idx + 1}/{len(labels_df)} records")

# Create the final dataframe
df = pd.DataFrame(records)

# Display information about the final dataframe
print("\nFinal DataFrame:")
print(df.head())


Tracings shape: (1631, 4096, 12)

Final DataFrame:
  exam_id  chagas                                                  I  \
0  247007    True  [-2.621980667114258, -2.6143863201141357, -2.6...   
1  181629    True  [4.146388053894043, 4.144881248474121, 4.13754...   
2  406936    True  [-0.22866110503673553, -0.2172079086303711, -0...   
3  253803    True  [0.0293155238032341, 0.03249815106391907, 0.06...   
4  253214    True  [0.08384239673614502, 0.08182810246944427, 0.0...   

                                                  II  \
0  [4.466513156890869, 4.469058990478516, 4.47073...   
1  [5.67079496383667, 5.662312984466553, 5.651445...   
2  [0.21341702342033386, 0.2776165306568146, 0.22...   
3  [0.0293155238032341, 0.04724210128188133, 0.02...   
4  [0.27439332008361816, 0.28447631001472473, 0.3...   

                                                 III  \
0  [7.088493824005127, 7.083445072174072, 7.08068...   
1  [1.5244072675704956, 1.5174311399459839, 1.513...   
2  [0.4420

In [ ]:
# File paths
hdf5_file_path = "../data/PTB/records500"
labels_csv_path = "../data/PTB/ptbxl_database.csv"

# Load the labels dataframe and display it first
labels_df = pd.read_csv(labels_csv_path)

# Initialize the records list to store processed data
records = []

# Define lead names in the correct order
lead_names = ['I', 'II', 'III', 'AVR', 'AVL', 'AVF',
              'V1', 'V2', 'V3', 'V4', 'V5', 'V6']

# Open HDF5 file and process data
with h5py.File(hdf5_file_path, 'r') as hdf:
    # Access the tracings dataset
    tracings = hdf['tracings']
    print(f"\nTracings shape: {tracings.shape}")
    
    # Process each row in the labels dataframe
    for idx, row in labels_df.iterrows():
        # Get information from CSV
        exam_id = str(row['exam_id'])
        chagas_val = row['chagas']
        
        # Use the index position to access the corresponding tracing
        ecg_data = tracings[idx, :, :]  # shape: (4096, 12)
        
        # Create a record with exam_id and chagas label
        record = {"exam_id": exam_id, "chagas": chagas_val}
        
        # Process each lead
        for j, lead in enumerate(lead_names):
            signal = ecg_data[:, j]
            
            # Find indices of non-zero elements for trimming
            nonzero_indices = np.nonzero(signal)[0]
            if nonzero_indices.size > 0:
                start = nonzero_indices[0]
                end = nonzero_indices[-1] + 1  # +1 to include the last non-zero element
                trimmed_signal = signal[start:end]
            else:
                # If the signal is all zeros, keep it as is
                trimmed_signal = signal
                
            record[lead] = trimmed_signal
        
        records.append(record)
        
        # # Print progress every 100 records
        # if (idx + 1) % 100 == 0:
        #     print(f"Processed {idx + 1}/{len(labels_df)} records")

# Create the final dataframe
df = pd.DataFrame(records)

# Display information about the final dataframe
print("\nFinal DataFrame:")
print(df.head())